In [1]:
%load_ext autoreload
%autoreload 2

import re
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib
plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 12, 6
matplotlib.rcParams['font.size'] = 18  # Probably OS Dependent

In [3]:
data_files = glob('../public/*.json')

dfs = []
for file_ in data_files:
    df = pd.read_json(file_)
    df['state'] = re.search(r'public[/\\](.+).json', file_).group(1).title()
    dfs += [df]
    
df = pd.concat(dfs).reset_index()

def empty_null(el_list):
    if type(el_list) != list:
        return []
    return el_list
df['emails'] = df['emails'].apply(empty_null)
df['faxes'] = df['faxes'].apply(empty_null)
df['phones'] = df['phones'].apply(empty_null)

df.sample(5, random_state=42)

,index,locale,official,emails,faxes,phones,county,address,party,state,city,url,officialTitle,physicalAddress,other_officials,contacts,urls,fipscode
5061,44,Marshall County,NaN,[MarshallCounty@elections.ok.gov],[580-795-5460],[580-795-5460],Marshall County,"PO Box 9, Madill OK 73446-0009",NaN,Oklahoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1557,939,Town of Monroe:Green County,Peggy Murphy,[townofmonroe@tds.net],[],[],Green County,"W5445 Center Road, Monroe, Wi 53566",NaN,Wisconsin,Town of Monroe,NaN,NaN,"N3302 Monroe Rd, Monroe Wi 53566",NaN,NaN,NaN,NaN
3922,507,Forest Home Township:Antrim County,Suzanne Mahan,[suemahan@yahoo.com],[(213) 533-5067],[(231) 533-8003],Antrim County,"321 N Bridge St, P O Box 317, Bellaire, Michig...",NaN,Michigan,Forest Home Township,NaN,NaN,NaN,NaN,NaN,NaN,29600.0
1103,485,Town of Dover:Buffalo County,Jennifer Severson,[jen_severson@nelson-tel.net],[],[],Buffalo County,"S891 County Road Bb, Independence, Wi 54747-8015",NaN,Wisconsin,Town of Dover,NaN,NaN,"S1194 Theisen Ridge Rd, Mondovi, Wi 54755-8359",NaN,NaN,NaN,NaN
4568,1153,Port Sheldon Township:Ottawa County,Teresa De Graaf,[teresa@portsheldontwp.org],[],[(616) 399-6121],Ottawa County,"16201 Port Sheldon St, West Olive, Michigan 4...",NaN,Michigan,Port Sheldon Township,NaN,NaN,NaN,NaN,NaN,NaN,65940.0


In [4]:
def non_empty_element(el_list):
    values = [len(x.strip()) > 0 for x in el_list if x]
    if len(values) == 0:
        return False
    return any(values)

def not_null_empty(series):
    return (series.notnull() & series.apply(lambda x: x != ''))

email = df['emails'].apply(empty_null).apply(non_empty_element)
fax = df['faxes'].apply(empty_null).apply(non_empty_element)
phone = df['phones'].apply(empty_null).apply(non_empty_element)
city = not_null_empty(df['city'])
county = not_null_empty(df['county'])

df_valid = pd.DataFrame({
    'official': df['official'].notnull(),
    'locale': not_null_empty(df['locale']),
    'city': city,
    'county': county,
    'city_county': (city | county),
    'email': email,
    'faxes': fax,
    'phones': phone,
    'fax_email': (email | fax),
    'url': df['url'].notnull()
})

df_stat = df_valid.groupby(df['state']).mean()
df_stat.loc[:, 'records'] = df.groupby('state').size()

df_len = df[['state']]
df_len.loc[:, 'emails'] = df['emails'].apply(lambda x: len(x))
df_len.loc[:, 'faxes'] = df['faxes'].apply(lambda x: len(x))
df_len.loc[:, 'phones'] = df['phones'].apply(lambda x: len(x))
df_stat.loc[:, 'email_max'] = df_len.groupby('state')['emails'].max()
df_stat.loc[:, 'fax_max'] = df_len.groupby('state')['faxes'].max()
df_stat.loc[:, 'phone_max'] = df_len.groupby('state')['phones'].max()
df_stat.sort_index()

/Users/tianhui/projects/elections-officials/venv/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/tianhui/projects/elections-officials/venv/lib/python3.7/site-packages/pandas/core/indexing.py:964: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,official,locale,city,county,city_county,email,faxes,phones,fax_email,url,records,email_max,fax_max,phone_max
state,,,,,,,,,,,,,,
Alaska,0.000000,1.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.000000,1,3,3,2
Arizona,1.000000,1.0,0.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,15,3,2,4
Florida,1.000000,1.0,0.000000,1.000000,1.0,1.000000,0.000000,0.000000,1.000000,1.000000,67,1,0,0
Georgia,1.000000,1.0,0.000000,1.000000,1.0,0.968553,0.987421,0.000000,0.993711,0.000000,159,1,1,0
Maine,1.000000,1.0,1.000000,0.000000,1.0,0.000000,0.942460,1.000000,0.942460,0.000000,504,0,1,1
Maryland,0.000000,1.0,0.041667,0.958333,1.0,1.000000,1.000000,0.916667,1.000000,1.000000,24,2,1,2
Massachusetts,0.000000,1.0,1.000000,0.000000,1.0,0.977208,0.997151,1.000000,1.000000,0.977208,351,1,1,1
Michigan,1.000000,1.0,1.000000,1.000000,1.0,0.997394,0.688599,1.000000,0.998697,0.000000,1535,1,1,1
Minnesota,1.000000,1.0,0.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.000000,87,1,1,1


## Michigan

In [5]:
df_mi = df[df['state'] == 'Michigan']
df_mi['city'].value_counts().head()

Grant Township       11
Sherman Township      9
Lincoln Township      7
Garfield Township     6
Lake Township         6
Name: city, dtype: int64

However, locales are unique

In [6]:
assert((df_mi['city'] + ':' + df_mi['county'].fillna('') == df_mi['locale']).all())
df_mi['locale'].value_counts().head()

Benton Township:Cheboygan County      1
Milton Township:Cass County           1
Fruitland Township:Muskegon County    1
Manton City:Wexford County            1
Reno Township:Iosco County            1
Name: locale, dtype: int64

## Wisconsin Unique Identifier

Wisonsin towns cannot be uniquely identified by city

In [7]:
df_wi = df[df['state'] == 'Wisconsin']
df_wi['city'].value_counts().head()

Town of Lincoln       12
Town of Washington     8
Town of Union          7
Town of Scott          7
Town of Grant          6
Name: city, dtype: int64

The locales are unique, but don't properly handle multi-county cities (blank entry)

In [8]:
assert((df_wi['city'] + ':' + df_wi['county'].fillna('') == df_wi['locale']).all())
df_wi['locale'].value_counts().head()

Town of Hancock:Waushara County                    1
Village of Fairwater:Fond Du Lac County            1
Town of Lakeside:Douglas County                    1
Town of La Prairie:Rock County                     1
Village of North Fond Du Lac:Fond Du Lac County    1
Name: locale, dtype: int64

So we need to ensure that all multi-county cities are unique across the entire state

In [9]:
multi_county_cities = df_wi.loc[df_wi['county'].isnull(), 'city']
df_wi.loc[df_wi['city'].isin(multi_county_cities), 'city'].value_counts().head()

Village of De Soto         1
Village of Lac La Belle    1
Village of Muscoda         1
City of Marshfield         1
City of Waupun             1
Name: city, dtype: int64